In [ ]:
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
from scipy.stats import norm
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

# Lecture 26

## Average (Mean) 

In [ ]:
values = make_array(2, 3, 3, 10)

In [ ]:
# use np.average() 


In [ ]:
# use np.mean() 


In [ ]:
# calculate manually


In [ ]:
# calculate manually 2


In [ ]:
# use np.median() to calculate median 


In [ ]:
# use percentile() to calculate median


In [ ]:
# load nba player data



In [ ]:
# visualize the heights of NBA players


In [ ]:
# calculate the median





In [ ]:
# calculate the mean


## Chebyshev's Bounds ##

In [ ]:
# get information on baby births
births = Table.read_table('baby.csv')
births

In [ ]:
# get the mean and sd of Maternal Pregnancy Weight






In [ ]:
# get the points that are within +/- 3 SD


In [ ]:
# calculate the proportion of points that are within +/- 3 SD


In [ ]:
# calculate Chebyshev's bounds


In [ ]:
# See if Chebyshev's bounds work for different distributions

for k in births.labels:
    values = births.column(k)
    mean = np.mean(values)
    sd = np.std(values)
    print()
    print(k)
    for z in np.arange(2, 6):
        chosen = births.where(k, are.between(mean - z*sd, mean + z*sd))
        proportion = chosen.num_rows/births.num_rows
        percent = round(proportion * 100, 2)
        print('Average plus or minus', z, 'SDs:', percent, "%   Chebyshev's bound:", np.round(100 * (1 - 1/z**2)), 2)
        

## Standard Units ##

In [ ]:
# Load LeBron James' statistics and the league statistics
bb_stats = Table.read_table('LeBron.csv')
bb_stats

In [ ]:
# calculate z-scores



In [ ]:
# define a function that can calculate z-scoes for all values in an array
def standard_units(x):
    """Convert array x to standard units."""


In [ ]:
# get the mother's ages


In [ ]:
# calculate z-scores for the mother's ages



In [ ]:
# what are the mean and sd of the z-scored data?


In [ ]:
# display z-scores for each age





In [ ]:
# mean and sd of original data


In [ ]:
# histogram of original data


In [ ]:
# z-score only changes the units, not the shape of the distribution



## The normal curve ##

In [ ]:
# The normal curve






### Data is often normal

In [ ]:
# heights are normally distributed


In [ ]:
# weights are also normally distributed


In [ ]:
# get the average and sd of weights





In [ ]:
# calculate the values of mean +/- 2 * SD


In [ ]:
# calculate percentiles for the middle 95% of the data


## Central Limit Theorem and Simulating Sample Mean ##

In [ ]:
# get the data set of flight delays
united = Table.read_table('united_summer2015.csv')
united

In [ ]:
# visualize the data (which is not normal!)


In [ ]:
# calculate the mean and sd of the delays






In [ ]:
# Take random sample from population of size sample_size 
# Repeat to get empirical distribution of sample average








In [ ]:
# visualize the distribution of sample means





In [ ]:
# look at the mean, of our means array


In [ ]:
# look at the mean of the original data


In [ ]:
# look at the original SD divided by square root of n
